<a href="https://colab.research.google.com/github/Shereen74/graduation_project/blob/main/Copy_of_3rd_Copy_of_PredictingEnergyConsumption_GP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%cd /content/drive/MyDrive/GP

/content/drive/MyDrive/GP


In [5]:
!pwd

/content/drive/MyDrive/GP


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

weather_data = pd.read_csv('./ashrae-energy-prediction/weather_train.csv')
building_data = pd.read_csv('./ashrae-energy-prediction/building_metadata.csv')

In [ ]:
counter = 0
batch_size = 1000000
train_data = []
for chunk in pd.read_csv('./ashrae-energy-prediction/train.csv', chunksize=batch_size):

    train_data.append(chunk)
    counter += 1

train_data

[        building_id  meter            timestamp  meter_reading
 0                 0      0  2016-01-01 00:00:00         0.0000
 1                 1      0  2016-01-01 00:00:00         0.0000
 2                 2      0  2016-01-01 00:00:00         0.0000
 3                 3      0  2016-01-01 00:00:00         0.0000
 4                 4      0  2016-01-01 00:00:00         0.0000
 ...             ...    ...                  ...            ...
 999995         1244      1  2016-01-19 03:00:00        22.9297
 999996         1244      3  2016-01-19 03:00:00      1050.9600
 999997         1245      0  2016-01-19 03:00:00        65.6562
 999998         1245      1  2016-01-19 03:00:00         0.0000
 999999         1245      2  2016-01-19 03:00:00       867.3520
 
 [1000000 rows x 4 columns],
          building_id  meter            timestamp  meter_reading
 1000000         1246      0  2016-01-19 03:00:00        49.0000
 1000001         1246      1  2016-01-19 03:00:00         0.0000
 10000

In [ ]:
Train_Data = pd.concat(train_data)
Train_Data.shape

(20216100, 4)

In [ ]:
Train_Data.tail()

,building_id,meter,timestamp,meter_reading
20216095,1444,0,2016-12-31 23:00:00,8.750
20216096,1445,0,2016-12-31 23:00:00,4.825
20216097,1446,0,2016-12-31 23:00:00,0.000
20216098,1447,0,2016-12-31 23:00:00,159.575
20216099,1448,0,2016-12-31 23:00:00,2.850


In [ ]:
weather_data.tail()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
139768,15,2016-12-31 19:00:00,3.0,NaN,-8.0,NaN,NaN,180.0,5.7
139769,15,2016-12-31 20:00:00,2.8,2.0,-8.9,NaN,1007.4,180.0,7.7
139770,15,2016-12-31 21:00:00,2.8,NaN,-7.2,NaN,1007.5,180.0,5.1
139771,15,2016-12-31 22:00:00,2.2,NaN,-6.7,NaN,1008.0,170.0,4.6
139772,15,2016-12-31 23:00:00,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8


## Data Preprocessing

In [ ]:
#Merging train data
train = Train_Data.merge(building_data, on='building_id', how='left')
train = train.merge(weather_data, on=['site_id', 'timestamp'], how='left')
train.tail(25)

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
20216075,1431,0,2016-12-31 23:00:00,87.9500,15,Public services,111360,2000.0,NaN,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8
20216076,1431,2,2016-12-31 23:00:00,426.3400,15,Public services,111360,2000.0,NaN,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8
20216077,1432,0,2016-12-31 23:00:00,403.4500,15,Education,160673,1968.0,NaN,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8
20216078,1433,0,2016-12-31 23:00:00,41.3500,15,Education,28084,1913.0,NaN,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8
20216079,1433,2,2016-12-31 23:00:00,3173.8800,15,Education,28084,1913.0,NaN,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8
20216080,1434,0,2016-12-31 23:00:00,70.7250,15,Education,33148,1967.0,NaN,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8
20216081,1434,2,2016-12-31 23:00:00,259.0730,15,Education,33148,1967.0,NaN,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8
20216082,1435,0,2016-12-31 23:00:00,4.7250,15,Education,9552,1961.0,NaN,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8
20216083,1436,0,2016-12-31 23:00:00,11.6000,15,Manufacturing/industrial,11302,1937.0,NaN,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8
20216084,1436,2,2016-12-31 23:00:00,1274.6600,15,Manufacturing/industrial,11302,1937.0,NaN,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8


In [ ]:
#Data correction for site 0, electric meter type which is 0
mask = (train['site_id'] == 0) & (train['meter'] == 0)

# Update the values in the "meter_reading" column for the selected rows
train.loc[mask, 'meter_reading'] *= 0.2931

In [ ]:
train.describe()

,building_id,meter,meter_reading,site_id,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
count,2.021610e+07,2.021610e+07,2.021610e+07,2.021610e+07,2.021610e+07,8.088455e+06,3.506933e+06,2.011944e+07,1.139074e+07,2.011596e+07,1.646708e+07,1.898443e+07,1.876705e+07,2.007242e+07
mean,7.992780e+02,6.624412e-01,2.109925e+03,7.992232e+00,1.077830e+05,1.968277e+03,4.184848e+00,1.598795e+01,1.900423e+00,7.747429e+00,7.964155e-01,1.016085e+03,1.730151e+02,3.377525e+00
std,4.269133e+02,9.309921e-01,1.532357e+05,5.099060e+00,1.171424e+05,3.020815e+01,4.008277e+00,1.094729e+01,2.402909e+00,1.017867e+01,7.468997e+00,7.060539e+00,1.140574e+02,2.265694e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.830000e+02,1.900000e+03,1.000000e+00,-2.890000e+01,0.000000e+00,-3.500000e+01,-1.000000e+00,9.682000e+02,0.000000e+00,0.000000e+00
25%,3.930000e+02,0.000000e+00,1.797590e+01,3.000000e+00,3.252700e+04,1.951000e+03,1.000000e+00,8.600000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.011600e+03,7.000000e+01,2.100000e+00
50%,8.950000e+02,0.000000e+00,7.509000e+01,9.000000e+00,7.270900e+04,1.969000e+03,3.000000e+00,1.670000e+01,0.000000e+00,8.900000e+00,0.000000e+00,1.016000e+03,1.800000e+02,3.100000e+00
75%,1.179000e+03,1.000000e+00,2.526000e+02,1.300000e+01,1.391130e+05,1.993000e+03,6.000000e+00,2.410000e+01,4.000000e+00,1.610000e+01,0.000000e+00,1.020500e+03,2.800000e+02,4.600000e+00
max,1.448000e+03,3.000000e+00,2.190470e+07,1.500000e+01,8.750000e+05,2.017000e+03,2.600000e+01,4.720000e+01,9.000000e+00,2.610000e+01,3.430000e+02,1.045500e+03,3.600000e+02,1.900000e+01


In [ ]:
100 * (train.isnull().sum() / len(train))

building_id            0.000000
meter                  0.000000
timestamp              0.000000
meter_reading          0.000000
site_id                0.000000
primary_use            0.000000
square_feet            0.000000
year_built            59.990033
floor_count           82.652772
air_temperature        0.478124
cloud_coverage        43.655131
dew_temperature        0.495348
precip_depth_1_hr     18.544739
sea_level_pressure     6.092515
wind_direction         7.167792
wind_speed             0.710701
dtype: float64

In [ ]:
#Dropping unused columns in test1 (decreasing null values)
train = train.drop(columns=['year_built', 'floor_count', 'wind_direction', 'dew_temperature'])

In [ ]:
#Replace Null values
#train_x['precip_depth_1_hr'].fillna(method='ffill', inplace = True)
#train_x['cloud_coverage'].fillna(method='bfill', inplace = True)
train['air_temperature'].fillna(train['air_temperature'].mean(), inplace = True)
train['cloud_coverage'].fillna(train['cloud_coverage'].mean(), inplace = True)
train['precip_depth_1_hr'].fillna(train['precip_depth_1_hr'].mean(), inplace = True)
train['sea_level_pressure'].fillna(train['sea_level_pressure'].mean(), inplace = True)
train['wind_speed'].fillna(train['wind_speed'].mean(), inplace = True)

100 * (train.isnull().sum() / len(train))

building_id           0.0
meter                 0.0
timestamp             0.0
meter_reading         0.0
site_id               0.0
primary_use           0.0
square_feet           0.0
air_temperature       0.0
cloud_coverage        0.0
precip_depth_1_hr     0.0
sea_level_pressure    0.0
wind_speed            0.0
dtype: float64

In [ ]:
#Displaying distribution of the meter reading in meters without zeros
#Used log transformation to avoid the skewness of the meter reading
# plt.figure(figsize=(10, 10))

# #Different meter types [0, 1, 2, 3]
# meterTypes = sorted(train['meter'].unique().tolist())

# #Plotting 'meter_reading' distribution for each meter
# for meter_type in meterTypes:
#     subset = train[train['meter'] == meter_type]
#     sns.kdeplot(np.log1p(subset["meter_reading"]), #estimates the density of a variable
#                 label = meter_type, linewidth = 2)  #and creates a smooth curve that represents the probability density of the data.

# #Setting figure information (label, title, legend)
# plt.ylabel("Density")
# plt.xlabel("Meter_reading")
# plt.title("Distribution graph for 'Meter Reading' ", size=13)
# plt.legend(['Electricity', 'Chilled water', 'Steam', 'Hot water'])

In [ ]:
train["primary_use"].value_counts()

Education                        8165504
Office                           4394864
Entertainment/public assembly    2264917
Lodging/residential              2146413
Public services                  1662826
Healthcare                        398527
Other                             242222
Parking                           213796
Manufacturing/industrial          125713
Food sales and service            114090
Retail                            112657
Warehouse/storage                 111861
Services                           96519
Technology/science                 77627
Utility                            56203
Religious worship                  32361
Name: primary_use, dtype: int64

In [ ]:
# Resample the train data by day and aggregating the meter readings for each building
train['timestamp'] = pd.to_datetime(train['timestamp'])

train['Month'] = pd.DatetimeIndex(train['timestamp']).month
train['Day'] = pd.DatetimeIndex(train['timestamp']).day

train_daily = train.groupby(['meter', 'building_id', 'primary_use', 'Month', 'Day', 'site_id']).agg({
    'meter_reading': 'sum',
    'air_temperature': 'mean',
    'wind_speed': 'mean',
    'precip_depth_1_hr': 'mean',
    'cloud_coverage': 'mean',
    'sea_level_pressure': 'mean',
    'square_feet': 'mean'
}).reset_index()

In [ ]:
# Resample the train data by month and aggregating the meter readings for each building
train['Year'] = pd.DatetimeIndex(train['timestamp']).year
train_monthly = train.groupby(['meter', 'building_id', 'primary_use', 'Month', 'Year', 'site_id']).agg({
    'meter_reading': 'sum',
    'air_temperature': 'mean',
    'wind_speed': 'mean',
    'precip_depth_1_hr': 'mean',
    'cloud_coverage': 'mean',
    'sea_level_pressure': 'mean',
    'square_feet': 'mean'
}).reset_index()

In [ ]:
# Use this function to continue the pre-processing for train_daily and train_monthly
# Call the function 2 times. After finishing, save the result of the function in a variable (Daily, Monthly)
def Preprocessing(train):

  #--------Encoding categorical data using Hot Encoder--------
  primary_use_one_hot_encoded = pd.get_dummies(train['primary_use'], prefix='primary_use')
  train = pd.concat([train, primary_use_one_hot_encoded], axis=1)
  train.pop('primary_use')

  # For simplicity, pop meter_reading column from train and saving it to a new variable, then add it as a last column
  meter_reading = train.pop('meter_reading')
  train = train.assign(meter_reading=meter_reading)

  #--------Scaling features using normalization--------
  from sklearn.preprocessing import StandardScaler
  scale = StandardScaler()
  train['air_temperature'] = train['air_temperature'].astype('float32')
  train['square_feet'] = train['square_feet'].astype('float32')
  train['sea_level_pressure'] = train['sea_level_pressure'].astype('float32')
  train['wind_speed'] = train['wind_speed'].astype('float32')
  train['precip_depth_1_hr'] = train['precip_depth_1_hr'].astype('float32')

  train.loc[:, ['square_feet', 'air_temperature', 'sea_level_pressure', 'wind_speed', 'precip_depth_1_hr']] = scale.fit_transform(train.loc[:, ['square_feet', 'air_temperature', 'sea_level_pressure', 'wind_speed', 'precip_depth_1_hr']])

  return train


In [ ]:
train_daily = Preprocessing(train_daily)

<ipython-input-18-ad8390e95f78>:23: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  train.loc[:, ['square_feet', 'air_temperature', 'sea_level_pressure', 'wind_speed', 'precip_depth_1_hr']] = scale.fit_transform(train.loc[:, ['square_feet', 'air_temperature', 'sea_level_pressure', 'wind_speed', 'precip_depth_1_hr']])


In [ ]:
train_monthly = Preprocessing(train_monthly)
train_monthly

<ipython-input-18-ad8390e95f78>:23: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  train.loc[:, ['square_feet', 'air_temperature', 'sea_level_pressure', 'wind_speed', 'precip_depth_1_hr']] = scale.fit_transform(train.loc[:, ['square_feet', 'air_temperature', 'sea_level_pressure', 'wind_speed', 'precip_depth_1_hr']])


,meter,building_id,Month,Year,site_id,air_temperature,wind_speed,precip_depth_1_hr,cloud_coverage,sea_level_pressure,...,primary_use_Other,primary_use_Parking,primary_use_Public services,primary_use_Religious worship,primary_use_Retail,primary_use_Services,primary_use_Technology/science,primary_use_Utility,primary_use_Warehouse/storage,meter_reading
0,0,0,1,2016,0,-0.113966,0.271216,1.572566,2.615440,0.638920,...,0,0,0,0,0,0,0,0,0,3.921145e+01
1,0,0,2,2016,0,0.033053,0.717058,-0.325942,2.160070,1.287959,...,0,0,0,0,0,0,0,0,0,0.000000e+00
2,0,0,3,2016,0,0.563755,0.261177,1.485960,2.520930,0.826973,...,0,0,0,0,0,0,0,0,0,0.000000e+00
3,0,0,4,2016,0,0.684353,0.397250,-0.951655,2.598216,0.365237,...,0,0,0,0,0,0,0,0,0,1.740507e+01
4,0,0,5,2016,0,0.922760,-0.186807,0.655920,2.444243,0.025128,...,0,0,0,0,0,0,0,0,0,1.701644e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28327,3,1331,8,2016,15,0.581542,-0.541897,1.126520,1.842573,0.364822,...,0,0,0,0,0,0,0,0,0,7.909817e+05
28328,3,1331,9,2016,15,0.149775,-0.769397,1.485227,1.359990,1.213176,...,0,0,0,0,0,0,0,0,0,1.360374e+06
28329,3,1331,10,2016,15,-0.531009,0.368906,2.874617,1.529977,1.127152,...,0,0,0,0,0,0,0,0,0,3.095771e+06
28330,3,1331,11,2016,15,-1.055443,0.587239,0.166784,1.399494,0.500577,...,0,0,0,0,0,0,0,0,0,4.146745e+06


In [ ]:
# Changing the format of 'meter_reading' for more clarity
train_monthly['meter_reading'] = train_monthly['meter_reading'].apply('{:.2f}'.format)
train_monthly

,meter,building_id,Month,Year,site_id,air_temperature,wind_speed,precip_depth_1_hr,cloud_coverage,sea_level_pressure,...,primary_use_Other,primary_use_Parking,primary_use_Public services,primary_use_Religious worship,primary_use_Retail,primary_use_Services,primary_use_Technology/science,primary_use_Utility,primary_use_Warehouse/storage,meter_reading
0,0,0,1,2016,0,-0.113966,0.271216,1.572566,2.615440,0.638920,...,0,0,0,0,0,0,0,0,0,39.21
1,0,0,2,2016,0,0.033053,0.717058,-0.325942,2.160070,1.287959,...,0,0,0,0,0,0,0,0,0,0.00
2,0,0,3,2016,0,0.563755,0.261177,1.485960,2.520930,0.826973,...,0,0,0,0,0,0,0,0,0,0.00
3,0,0,4,2016,0,0.684353,0.397250,-0.951655,2.598216,0.365237,...,0,0,0,0,0,0,0,0,0,17.41
4,0,0,5,2016,0,0.922760,-0.186807,0.655920,2.444243,0.025128,...,0,0,0,0,0,0,0,0,0,17016.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28327,3,1331,8,2016,15,0.581542,-0.541897,1.126520,1.842573,0.364822,...,0,0,0,0,0,0,0,0,0,790981.68
28328,3,1331,9,2016,15,0.149775,-0.769397,1.485227,1.359990,1.213176,...,0,0,0,0,0,0,0,0,0,1360374.11
28329,3,1331,10,2016,15,-0.531009,0.368906,2.874617,1.529977,1.127152,...,0,0,0,0,0,0,0,0,0,3095771.42
28330,3,1331,11,2016,15,-1.055443,0.587239,0.166784,1.399494,0.500577,...,0,0,0,0,0,0,0,0,0,4146745.18


In [ ]:
train_monthly[train_monthly['building_id'] == 1300]

,meter,building_id,Month,Year,site_id,air_temperature,wind_speed,precip_depth_1_hr,cloud_coverage,sea_level_pressure,...,primary_use_Other,primary_use_Parking,primary_use_Public services,primary_use_Religious worship,primary_use_Retail,primary_use_Services,primary_use_Technology/science,primary_use_Utility,primary_use_Warehouse/storage,meter_reading
15173,0,1300,1,2016,14,-1.640965,0.031259,-0.518457,1.138984,-0.073107,...,0,0,0,0,0,0,0,0,0,39517.60
15174,0,1300,2,2016,14,-1.383134,0.346117,0.467719,1.188711,-0.029443,...,0,0,0,0,0,0,0,0,0,39538.90
15175,0,1300,3,2016,14,-0.716124,0.323562,-0.528639,1.011470,0.062569,...,0,0,0,0,0,0,0,0,0,43013.70
15176,0,1300,4,2016,14,-0.480265,0.043902,-0.537060,0.893214,-0.014308,...,0,0,0,0,0,0,0,0,0,40403.21
15177,0,1300,5,2016,14,0.031675,-0.627727,0.422405,1.310036,-0.531663,...,0,0,0,0,0,0,0,0,0,42048.82
15178,0,1300,6,2016,14,0.661782,-0.517044,-0.638899,1.063327,-0.961155,...,0,0,0,0,0,0,0,0,0,42124.58
15179,0,1300,7,2016,14,0.993539,-1.000748,1.280361,0.977817,-0.735147,...,0,0,0,0,0,0,0,0,0,43543.10
15180,0,1300,8,2016,14,0.990979,-0.975191,-0.764277,1.088835,0.173607,...,0,0,0,0,0,0,0,0,0,41961.71
15181,0,1300,9,2016,14,0.581122,-0.436823,-0.269437,0.984581,0.645618,...,0,0,0,0,0,0,0,0,0,37109.82
15182,0,1300,10,2016,14,-0.177742,-0.506707,-0.220823,1.090341,0.954055,...,0,0,0,0,0,0,0,0,0,15405.08


In [ ]:
train_daily

,meter,building_id,Month,Day,site_id,air_temperature,wind_speed,precip_depth_1_hr,cloud_coverage,sea_level_pressure,...,primary_use_Other,primary_use_Parking,primary_use_Public services,primary_use_Religious worship,primary_use_Retail,primary_use_Services,primary_use_Technology/science,primary_use_Utility,primary_use_Warehouse/storage,meter_reading
0,0,0,1,1,0,0.712774,-0.963826,-0.410977,2.596133,0.422406,...,0,0,0,0,0,0,0,0,0,0.00
1,0,0,1,2,0,0.344816,0.346012,-0.388825,2.841984,0.427575,...,0,0,0,0,0,0,0,0,0,0.00
2,0,0,1,3,0,-0.111097,1.025969,-0.131158,2.917019,0.179512,...,0,0,0,0,0,0,0,0,0,0.00
3,0,0,1,4,0,-0.320494,0.572664,-0.057538,2.208510,-0.068730,...,0,0,0,0,0,0,0,0,0,0.00
4,0,0,1,5,0,-0.185737,1.821887,-0.352016,1.475106,0.975027,...,0,0,0,0,0,0,0,0,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845696,3,1331,12,27,15,-1.102404,1.431835,0.239366,2.167019,-0.557832,...,0,0,0,0,0,0,0,0,0,192295.66
845697,3,1331,12,28,15,-1.616061,0.659043,0.070306,1.900423,0.335166,...,0,0,0,0,0,0,0,0,0,233428.37
845698,3,1331,12,29,15,-1.654745,0.670178,-0.146805,1.591984,-1.100020,...,0,0,0,0,0,0,0,0,0,231253.90
845699,3,1331,12,30,15,-1.661603,1.611048,-0.396979,1.821239,-2.031877,...,0,0,0,0,0,0,0,0,0,252862.20


In [ ]:
# Split the dataset into train_daily and test_daily sets based on the "month" column
# Get the maximum month value in the "month" column
max_month = train_daily['Month'].max()

# Define the cutoff month for the test_daily
test_cutoff = max_month - 3 #the cutoff would be 9

test_daily = train_daily[train_daily['Month'] >= test_cutoff] #Test 9, 10, 11, 12
train_daily = train_daily[train_daily['Month'] < test_cutoff] #Train 1..8

# Same steps for train & test monthly..
test_monthly = train_monthly[train_monthly['Month'] >= test_cutoff] #Test 9, 10, 11, 12
train_monthly = train_monthly[train_monthly['Month'] < test_cutoff] #Train 1..8

In [ ]:
#Splitting independent => features, and dependent => label variables (DAILY)
from sklearn.model_selection import train_test_split
train_daily_x = train_daily.iloc[:, 0:-1] #independent, features
train_daily_y = train_daily.iloc[:, -1] #dependent, label
test_daily_x = test_daily.iloc[:, 0:-1] #independent, features
test_daily_y = test_daily.iloc[:, -1] #dependent, label

#Splitting independent => features, and dependent => label variables (MONTHLY)
train_monthly_x = train_monthly.iloc[:, 0:-1] #independent, features
train_monthly_y = train_monthly.iloc[:, -1] #dependent, label
test_monthly_x = test_monthly.iloc[:, 0:-1] #independent, features
test_monthly_y = test_monthly.iloc[:, -1] #dependent, label

In [7]:
#Converting train and test preprocessed data into CSV
# Comment after running this code
# train_daily_x.to_csv('./train_daily_x.csv', index=False)
# train_daily_y.to_csv('./train_daily_y.csv', index=False)
# test_daily_x.to_csv('./test_daily_x.csv', index=False)
# test_daily_y.to_csv('./test_daily_y.csv', index=False)

train_daily_x = pd.read_csv('./train_daily_x.csv', keep_default_na=False, index_col=None)
train_daily_y = pd.read_csv('./train_daily_y.csv', keep_default_na=False, index_col=None)
test_daily_x = pd.read_csv('./test_daily_x.csv', keep_default_na=False, index_col=None)
test_daily_y = pd.read_csv('./test_daily_y.csv', keep_default_na=False, index_col=None)

In [19]:
train_daily_x

,meter,building_id,Month,Day,site_id,air_temperature,wind_speed,precip_depth_1_hr,cloud_coverage,sea_level_pressure,...,primary_use_Office,primary_use_Other,primary_use_Parking,primary_use_Public services,primary_use_Religious worship,primary_use_Retail,primary_use_Services,primary_use_Technology/science,primary_use_Utility,primary_use_Warehouse/storage
0,0,0,1,1,0,0.712774,-0.963826,-0.410977,2.596133,0.422406,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,2,0,0.344816,0.346012,-0.388825,2.841984,0.427575,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,3,0,-0.111097,1.025969,-0.131158,2.917019,0.179512,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,4,0,-0.320494,0.572664,-0.057538,2.208510,-0.068730,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,5,0,-0.185737,1.821887,-0.352016,1.475106,0.975027,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558821,3,1331,8,27,15,0.505394,-1.469841,-0.000226,1.412518,0.948531,...,0,0,0,0,0,0,0,0,0,0
558822,3,1331,8,28,15,0.694973,-0.218575,0.831738,1.470957,0.885750,...,0,0,0,0,0,0,0,0,0,0
558823,3,1331,8,29,15,0.581600,-0.057810,-0.000226,2.096133,0.910950,...,0,0,0,0,0,0,0,0,0,0
558824,3,1331,8,30,15,0.155947,-1.214790,-0.000226,0.412518,0.905137,...,0,0,0,0,0,0,0,0,0,0


In [8]:
#Converting train and test preprocessed data into CSV
# Comment after running this code
# train_monthly_x.to_csv('./train_monthly_x.csv', index=False)
# train_monthly_y.to_csv('./train_monthly_y.csv', index=False)
# test_monthly_x.to_csv('./test_monthly_x.csv', index=False)
# test_monthly_y.to_csv('./test_monthly_y.csv', index=False)

train_monthly_x = pd.read_csv('./train_monthly_x.csv', keep_default_na=False, index_col=None)
train_monthly_y = pd.read_csv('./train_monthly_y.csv', keep_default_na=False, index_col=None)
test_monthly_x = pd.read_csv('./test_monthly_x.csv', keep_default_na=False, index_col=None)
test_monthly_y = pd.read_csv('./test_monthly_y.csv', keep_default_na=False, index_col=None)

# Data Modeling (LSTM)

In [10]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

In [33]:
# Reshape train_x data for LSTM to be 3D, it is reshaped to have a shape of (num_samples, time_steps/window_size, num_features)
# where:
# train_samples is the number of training samples (rows).
# window_size is the number of time steps or window size.
# train_x.shape[1] and train_y.shape[1] is the number of features in each time step.

def reshape_data_for_LSTM(train_x, train_y, test_x, test_y, window_size):
    train_samples = train_x.shape[0] - window_size + 1 #num of rows of features
    test_samples = test_x.shape[0] - window_size + 1 #num of rows of features

    # Reshape train inputs
    train_x_reshaped = np.zeros((train_samples, window_size, train_x.shape[1])) #Initializing the 3D array with zeros
    for i in range(train_samples): #We then use a loop to populate train_x_reshaped with a sequence of inputs by slicing train_x based on the window size.
        train_x_reshaped[i] = train_x[i:i+window_size]
    
    # Reshape train outputs
    train_y_reshaped = train_y[window_size-1:]

    # Reshape test inputs
    test_x_reshaped = np.zeros((test_samples, window_size, test_x.shape[1]))
    for i in range(test_samples):
        test_x_reshaped[i] = test_x[i:i+window_size]

    # Reshape test outputs
    test_y_reshaped = test_y[window_size-1:]
    
    # Convert to numpy arrays to be in the right format for the LSTM
    train_x_reshaped = np.array(train_x_reshaped)
    train_y_reshaped = np.array(train_y_reshaped)
    test_x_reshaped = np.array(test_x_reshaped)
    test_y_reshaped = np.array(test_y_reshaped)

    return train_x_reshaped, train_y_reshaped, test_x_reshaped, test_y_reshaped

In [32]:
train_daily_x

,meter,building_id,Month,Day,site_id,air_temperature,wind_speed,precip_depth_1_hr,cloud_coverage,sea_level_pressure,...,primary_use_Office,primary_use_Other,primary_use_Parking,primary_use_Public services,primary_use_Religious worship,primary_use_Retail,primary_use_Services,primary_use_Technology/science,primary_use_Utility,primary_use_Warehouse/storage
0,0,0,1,1,0,0.712774,-0.963826,-0.410977,2.596133,0.422406,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,2,0,0.344816,0.346012,-0.388825,2.841984,0.427575,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,3,0,-0.111097,1.025969,-0.131158,2.917019,0.179512,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,4,0,-0.320494,0.572664,-0.057538,2.208510,-0.068730,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,5,0,-0.185737,1.821887,-0.352016,1.475106,0.975027,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558821,3,1331,8,27,15,0.505394,-1.469841,-0.000226,1.412518,0.948531,...,0,0,0,0,0,0,0,0,0,0
558822,3,1331,8,28,15,0.694973,-0.218575,0.831738,1.470957,0.885750,...,0,0,0,0,0,0,0,0,0,0
558823,3,1331,8,29,15,0.581600,-0.057810,-0.000226,2.096133,0.910950,...,0,0,0,0,0,0,0,0,0,0
558824,3,1331,8,30,15,0.155947,-1.214790,-0.000226,0.412518,0.905137,...,0,0,0,0,0,0,0,0,0,0


In [50]:
window_size = 1;
train_daily_x_reshaped, train_daily_y_reshaped, test_daily_x_reshaped, test_daily_y_reshaped = reshape_data_for_LSTM(train_daily_x, train_daily_y, test_daily_x, test_daily_y, window_size)

In [51]:
train_monthly_x_reshaped, train_monthly_y_reshaped, test_monthly_x_reshaped, test_monthly_y_reshaped = reshape_data_for_LSTM(train_monthly_x, train_monthly_y, test_monthly_x, test_monthly_y, window_size)

In [52]:
train_monthly_x_reshaped.shape

(18847, 1, 27)

In [49]:
train_x_reshaped.shape

(558826, 1, 27)

In [45]:
train_daily_x

,meter,building_id,Month,Day,site_id,air_temperature,wind_speed,precip_depth_1_hr,cloud_coverage,sea_level_pressure,...,primary_use_Office,primary_use_Other,primary_use_Parking,primary_use_Public services,primary_use_Religious worship,primary_use_Retail,primary_use_Services,primary_use_Technology/science,primary_use_Utility,primary_use_Warehouse/storage
0,0,0,1,1,0,0.712774,-0.963826,-0.410977,2.596133,0.422406,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,2,0,0.344816,0.346012,-0.388825,2.841984,0.427575,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,3,0,-0.111097,1.025969,-0.131158,2.917019,0.179512,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,4,0,-0.320494,0.572664,-0.057538,2.208510,-0.068730,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,5,0,-0.185737,1.821887,-0.352016,1.475106,0.975027,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558821,3,1331,8,27,15,0.505394,-1.469841,-0.000226,1.412518,0.948531,...,0,0,0,0,0,0,0,0,0,0
558822,3,1331,8,28,15,0.694973,-0.218575,0.831738,1.470957,0.885750,...,0,0,0,0,0,0,0,0,0,0
558823,3,1331,8,29,15,0.581600,-0.057810,-0.000226,2.096133,0.910950,...,0,0,0,0,0,0,0,0,0,0
558824,3,1331,8,30,15,0.155947,-1.214790,-0.000226,0.412518,0.905137,...,0,0,0,0,0,0,0,0,0,0


In [39]:
test_x_reshaped.shape

(286866, 10, 27)

In [42]:
test_x_reshaped.shape

(286866, 10, 27)

In [43]:
test_daily_x

,meter,building_id,Month,Day,site_id,air_temperature,wind_speed,precip_depth_1_hr,cloud_coverage,sea_level_pressure,...,primary_use_Office,primary_use_Other,primary_use_Parking,primary_use_Public services,primary_use_Religious worship,primary_use_Retail,primary_use_Services,primary_use_Technology/science,primary_use_Utility,primary_use_Warehouse/storage
0,0,0,9,1,0,0.923382,0.282761,1.930183,2.917019,-0.448062,...,0,0,0,0,0,0,0,0,0,0
1,0,0,9,2,0,1.036755,2.080165,2.022207,2.833686,-0.541768,...,0,0,0,0,0,0,0,0,0,0
2,0,0,9,3,0,1.021423,-0.326038,-0.075943,2.929466,-0.002817,...,0,0,0,0,0,0,0,0,0,0
3,0,0,9,4,0,1.141252,-1.172030,5.445506,2.954360,0.309408,...,0,0,0,0,0,0,0,0,0,0
4,0,0,9,5,0,0.913295,0.187883,4.801337,2.762800,0.513614,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286870,3,1331,12,27,15,-1.102404,1.431835,0.239366,2.167019,-0.557832,...,0,0,0,0,0,0,0,0,0,0
286871,3,1331,12,28,15,-1.616061,0.659043,0.070306,1.900423,0.335166,...,0,0,0,0,0,0,0,0,0,0
286872,3,1331,12,29,15,-1.654745,0.670178,-0.146805,1.591984,-1.100020,...,0,0,0,0,0,0,0,0,0,0
286873,3,1331,12,30,15,-1.661603,1.611048,-0.396979,1.821239,-2.031877,...,0,0,0,0,0,0,0,0,0,0


In [44]:
test_x_reshaped.shape

(286866, 10, 27)

In [36]:
train_y_reshaped.shape

(558817, 1)

In [ ]:
model=Sequential()
model.add(LSTM(32,activation='relu',input_shape=(trainX.shape[1],trainX.shape[2]),return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(32,activation='relu',return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(32,activation='relu',return_sequences=False))
model.add(Dropout(0.3))

model.add(Dense(trainY.shape[1]))

model.compile(optimizer='adam',loss='mse')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 20, 32)            5504      
                                                                 
 dropout (Dropout)           (None, 20, 32)            0         
                                                                 
 lstm_1 (LSTM)               (None, 20, 32)            8320      
                                                                 
 dropout_1 (Dropout)         (None, 20, 32)            0         
                                                                 
 lstm_2 (LSTM)               (None, 32)                8320      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 3

In [ ]:
history = model.fit(trainX, trainY, epochs=30, batch_size=5, validation_split=0.2, verbose=1)

In [ ]:
plt.plot(history.history['loss'],label='Training loss')
plt.plot(history.history['val_loss'],label='Validation Loss')
plt.legend()